# Kaggle : Airbnb New User Bookings

In [1]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import seaborn as sns

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

# data

# sessions

In [ ]:
#10M data
sessions = pd.read_csv("~/Documents/data/airbnb/sessions.csv")
sessions.head()

In [ ]:
sessions.isnull().sum()

In [ ]:
#359 action type
sessions['action'].value_counts()

In [ ]:
#155 action detail
sessions['action_detail'].value_counts()

In [ ]:
#10 action type
sessions['action_type'].value_counts()

In [ ]:
# sns.distplot(sessions['secs_elapsed']);
# plt.show()

# age_gender_bkts

In [ ]:
#21 age bucket (5 years) per gender by country
#1K unit
age_gender = pd.read_csv("~/Documents/data/airbnb/age_gender_bkts.csv")
age_gender.head()

In [ ]:
age_gender.info()

# countries (destination)

- AU = austrailia
- CA = canada
- DE = germany
- ES = spain
- FR = france
- GB = england
- IT = italia
- NL = netherland 
- PT = Portugal
- US = USA

In [ ]:
countries = pd.read_csv("~/Documents/data/airbnb/countries.csv")
countries

In [ ]:
countries['country_destination'].value_counts()

# submission

In [ ]:
# submission = pd.read_csv("~/Documents/data/airbnb/sample_submission_NDF.csv")
# submission.head()

In [ ]:
# submission.info()

# test

In [ ]:
test = pd.read_csv("~/Documents/data/airbnb/test_users.csv")
test.head()

In [ ]:
test.isnull().sum()

# train

In [ ]:
train = pd.read_csv("~/Documents/data/airbnb/train_users_2.csv")
train.head()

In [ ]:
train.isnull().sum()

# drop date_first_booking column

In [ ]:
train = train.drop(labels=['date_first_booking'], axis=1)
test = test.drop(labels=['date_first_booking'], axis=1)

# first_affiliate_tracked nan data manipulation

In [ ]:
train['first_affiliate_tracked'] = train.first_affiliate_tracked.replace(np.nan, "untracked")
test['first_affiliate_tracked'] = test.first_affiliate_tracked.replace(np.nan, "untracked")

# age

In [ ]:
print(len(train[train['age'] > 120]))

In [ ]:
train[train['age'] > 120]

In [ ]:
train_copy = train.copy()
train_copy = train_copy[train_copy['age'] <= 120]
train_copy.isnull().sum()

In [ ]:
train_copy = train.copy()
train_copy = train_copy.dropna()
train_copy = train_copy[train_copy['age'] <= 120]

ax1 = sns.distplot(train_copy['age'])
plt.show()
ax2 = sns.boxplot(x = train_copy['country_destination'], y = train_copy['age'], data = train_copy)
plt.show()
# ax3 = sns.swarmplot(x = train_copy['country_destination'], y = train_copy['age'], data = train_copy)
# plt.show()

# age nan value manipulation

- DBSCAN
- heatmap age vs related features

In [ ]:
# table = pivot_table(train, values='age', columns=['gender', 'signup_method', 'affiliate_provider', 'signup_app'])

In [ ]:
train_copy.describe()

In [ ]:
# #data type convert
# train_test["date_account_created"] =  pd.to_datetime(train_test["date_account_created"])
# train_test["date_first_booking"] =  pd.to_datetime(train_test["date_first_booking"])

# train_test["date_first_booking"] =  pd.to_datetime(train_test["date_first_booking"])
# train_test.info()

In [ ]:
# train_test = train_test.fillna(0)
# train_test.info()

In [ ]:
# vals = train_test.values
# X = vals[:piv_train]
# le = LabelEncoder()
# y = le.fit_transform(labels)   
# X_test = vals[piv_train:]

# random forest regression

In [ ]:
import numpy as np

from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score

rng = np.random.RandomState(0)

dataset = load_boston()
X_full, y_full = dataset.data, dataset.target
n_samples = X_full.shape[0]
n_features = X_full.shape[1]

# Estimate the score on the entire dataset, with no missing values
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_full, y_full).mean()
print("Score with the entire dataset = %.2f" % score)

# Add missing values in 75% of the lines
missing_rate = 0.75
n_missing_samples = int(np.floor(n_samples * missing_rate))
missing_samples = np.hstack((np.zeros(n_samples - n_missing_samples,
                                      dtype=np.bool),
                             np.ones(n_missing_samples,
                                     dtype=np.bool)))
rng.shuffle(missing_samples)
missing_features = rng.randint(0, n_features, n_missing_samples)

# Estimate the score without the lines containing missing values
X_filtered = X_full[~missing_samples, :]
y_filtered = y_full[~missing_samples]
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_filtered, y_filtered).mean()
print("Score without the samples containing missing values = %.2f" % score)

# Estimate the score after imputation of the missing values
X_missing = X_full.copy()
X_missing[np.where(missing_samples)[0], missing_features] = 0
y_missing = y_full.copy()
estimator = Pipeline([("imputer", Imputer(missing_values=0,
                                          strategy="mean",
                                          axis=0)),
                      ("forest", RandomForestRegressor(random_state=0,
                                                       n_estimators=100))])
score = cross_val_score(estimator, X_missing, y_missing).mean()
print("Score after imputation of the missing values = %.2f" % score)

In [ ]:
X_full1 = train.drop(labels=['age'], axis=1)
y_full1 = train['age']

In [ ]:
rng = np.random.RandomState(0)

dataset = load_boston()
X_full, y_full = dataset.data, dataset.target
n_samples = X_full.shape[0]
n_features = X_full.shape[1]

# Estimate the score on the entire dataset, with no missing values
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_full, y_full).mean()
print("Score with the entire dataset = %.2f" % score)

# Add missing values in 75% of the lines
missing_rate = 0.75
n_missing_samples = int(np.floor(n_samples * missing_rate))
missing_samples = np.hstack((np.zeros(n_samples - n_missing_samples,
                                      dtype=np.bool),
                             np.ones(n_missing_samples,
                                     dtype=np.bool)))
rng.shuffle(missing_samples)
missing_features = rng.randint(0, n_features, n_missing_samples)

# Estimate the score without the lines containing missing values
X_filtered = X_full[~missing_samples, :]
y_filtered = y_full[~missing_samples]
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_filtered, y_filtered).mean()
print("Score without the samples containing missing values = %.2f" % score)

# Estimate the score after imputation of the missing values
X_missing = X_full.copy()
X_missing[np.where(missing_samples)[0], missing_features] = 0
y_missing = y_full.copy()
estimator = Pipeline([("imputer", Imputer(missing_values=0,
                                          strategy="mean",
                                          axis=0)),
                      ("forest", RandomForestRegressor(random_state=0,
                                                       n_estimators=100))])
score = cross_val_score(estimator, X_missing, y_missing).mean()
print("Score after imputation of the missing values = %.2f" % score)

In [ ]:
# train_1 = train.drop(['country_destination'], axis=1)
# train_1.info()

In [ ]:
# y = train['country_destination']

In [ ]:
# X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
# regr = RandomForestRegressor(max_depth=2, random_state=0)
# regr.fit(train_1, y)

# print(regr.feature_importances_)
# print(regr.predict([[0, 0, 0, 0]]))


# nueral network

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [ ]:
clf.fit(train, train_no_target)

# pre processing

In [70]:
import pandas as pd

In [71]:
df_train = pd.read_csv("~/Documents/data/airbnb/train_users_2.csv")
df_test = pd.read_csv("~/Documents/data/airbnb/test_users.csv")
df_all = pd.concat([df_train, df_test])

In [72]:
df_target = df_train["country_destination"]

In [73]:
df_all = df_all.drop("age", axis = 1)
df_all = df_all.drop("date_first_booking", axis = 1)

In [74]:
df_all["first_affiliate_tracked"].fillna("untracked", inplace = True)

In [75]:
df_all.isnull().sum()

affiliate_channel              0
affiliate_provider             0
country_destination        62096
date_account_created           0
first_affiliate_tracked        0
first_browser                  0
first_device_type              0
gender                         0
id                             0
language                       0
signup_app                     0
signup_flow                    0
signup_method                  0
timestamp_first_active         0
dtype: int64

In [76]:
df_all["date_account_created"] = pd.to_datetime(df_all["date_account_created"], format = "%Y-%m-%d")

In [77]:
df_all["timestamp_first_active"] = pd.to_datetime(df_all["timestamp_first_active"], format="%Y%m%d%H%M%S", errors='ignore')

In [78]:
df_all["ac_year"] = df_all["date_account_created"].apply(lambda x : x.year)
df_all["ac_month"] = df_all["date_account_created"].apply(lambda x : x.month)
df_all["ac_day"] = df_all["date_account_created"].apply(lambda x : x.day)

In [79]:
df_all["fa_year"] = df_all["timestamp_first_active"].apply(lambda x : x.year)
df_all["fa_month"] = df_all["timestamp_first_active"].apply(lambda x : x.month)
df_all["fa_day"] = df_all["timestamp_first_active"].apply(lambda x : x.day)
df_all["fa_hour"] = df_all["timestamp_first_active"].apply(lambda x : x.hour)
df_all["fa_minute"] = df_all["timestamp_first_active"].apply(lambda x : x.minute)
df_all["fa_second"] = df_all["timestamp_first_active"].apply(lambda x : x.second)

In [80]:
df_all = df_all.drop("date_account_created", axis = 1)
df_all = df_all.drop("timestamp_first_active", axis = 1)
df_all = df_all.drop("country_destination", axis = 1)

In [81]:
df_all = df_all.drop("id", axis = 1)

In [82]:
df_all = pd.get_dummies(df_all)

In [83]:
df_train = df_all.iloc[:213451]
df_test = df_all.iloc[213451:]

In [89]:
df_train.shape, df_test.shape, df_target.shape

((213451, 145), (62096, 145), (213451,))

# XGBoost

In [85]:
import xgboost as xgb

In [86]:
data = np.random.rand(5, 10)  # 5 entities, each contains 10 features
label = np.random.randint(2, size=5)  # binary target

dtrain = xgb.DMatrix(df_train)
dtest = xgb.DMatrix(df_test)

In [95]:
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective' : 'multi:softmax'  , 'eval_metric' : 'ndcg'}
num_round = 2
bst = xgb.XGBModel(param, num_round).fit(df_train, df_target)
# make prediction
preds = bst.predict(df_test)

ValueError: could not convert string to float: 'NDF'

# QDA

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
y = df_target
X = df_train
X_test = df_test

In [ ]:
y.head()

In [ ]:
>>> from sklearn.datasets import load_boston
>>> from sklearn.model_selection import cross_val_score
>>> from sklearn.tree import DecisionTreeRegressor

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [ ]:
models_r = RandomForestClassifier(max_depth=2)
models_r.fit(X, y)

In [ ]:
y_pred_r = models_r.predict(X_test).astype('int')

In [ ]:
aaa_r = pd.DataFrame(y_pred_r)

In [ ]:
regressor_d = DecisionTreeRegressor(random_state=0, max_depth=2)

In [ ]:
regressor_d.fit(X,y)

In [ ]:
y_pred_d = regressor_d.predict(X_test).astype('int')

In [ ]:
aaa_d = pd.DataFrame(y_pred_d)

In [ ]:
len(y_pred_d), len(y_pred_r) 

In [ ]:
aaa_r.describe()

In [ ]:
aaa_d.describe()

In [ ]:
y_pred_r = pd.DataFrame(y_pred_r, columns=["age"], index=range(len(y_pred_r)))
df_target = pd.DataFrame(df_target)

In [ ]:
y_pred_r.tail()

In [ ]:
y_pred_r.tail()

In [ ]:
df_target.

In [ ]:
age = pd.concat([y_pred_r, df_target], ignore_index=True)
# age = age.drop(labels='age', axis=1)

In [ ]:
age.isnull().sum()

In [ ]:
age.isnull().sum()

In [ ]:
len(age)

In [ ]:
df_all['age'] = age

In [ ]:
df_all.head()

In [ ]:
df_all.info()

In [ ]:
df_all.isnull().sum()

# Cross validation

### k fold stacking

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
x

In [ ]:
kf = KFold(ntrian, n_folds=nfolds, shuffle = kfodl_shuffle, random_state = kfold_random_state)

oof_train = np.zeros((ntrain,))
oof_test = np.zeros((ntest,))
oof_test_skf = np.empty((nfolds, ntest))

In [ ]:
for i, (train_index, cross_index) in enumerate(kf):
    x_tr, x_cr = x_train.iloc[train_index], x_train.iloc[cross_index]
    y_tr, y_cr = y_train.iloc[train_index], y_train.iloc[cross_index]
    
    clf.train(x_tr, y_tr, x_cr, y_cr)

In [ ]:
oof_train[cross_index] = clf.predict(x_cr)
oof_test_skf[i, :] = clf.predict(x_test)
oof_train[:] = oof_test_skf.mean(axis=0)


In [3]:
import numpy as np

from sklearn.model_selection import cross_val_predict, KFold
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X = np.array([[1, 2], [2, 4], [3, 2], [4, 4], [5, 2], [6, 4], [7, 2], [8, 4], [1, 2], [2, 4], [3, 2], [4, 4], [5, 2], [6, 4], [7, 2], [8, 4]])
Y = np.array([1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8])

Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,train_size=0.5,random_state=1)

clf = LinearDiscriminantAnalysis()
clf.fit(X, Y)

# without cross-valdidation
prediction = clf.predict(Xtest)

# with cross-valdidation
cv = KFold(n_splits=2)
prediction_cv = cross_val_predict(clf, X, Y, cv=cv)
prediction_cv

/home/jk/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jk/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/jk/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/jk/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


array([5, 6, 6, 7, 7, 8, 8, 8, 1, 2, 2, 3, 3, 4, 4, 4])

In [ ]:
df_train1 = df_all.iloc[:213451]
df_test1 = df_all.iloc[213451:]

In [ ]:
df_train1.isnull().sum()

In [ ]:
y1 = y_destination
X1 = df_train1
X_test1 = df_test1

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
models_r = LinearDiscriminantAnalysis(n_components=3, solver='svd', store_covariance=True).fit(X1, y1)
# models_r.fit(X1, y1)

In [ ]:
y_total1 = models_r.predict(X_test1)

In [ ]:
len(y_total1)

In [ ]:
y_total1_df = pd.DataFrame(y_total1)
y_total1_df.tail()

In [ ]:
y_total1_df[0].value_counts()

In [ ]:
import xgboost
from xgboost.sklearn import XGBClassifier


In [ ]:
x = X1
y = y1

In [ ]:
odels_xgb = XGBClassifier().fit(x, y)
# models_r.fit(X1, y1)

In [ ]:
y_XGB = odels_xgb.predict(X_test1)

In [ ]:
y_XGB11 = pd.DataFrame(y_XGB)
y_XGB11.tail()

In [ ]:
y_XGB11[0].value_counts()